In [1]:
!pip install catalyst
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install efficientnet_pytorch
!pip install timm
!pip install torchtoolbox

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-kc22jrzz
  Running command git clone --filter=blob:none -q https://github.com/albu/albumentations /tmp/pip-req-build-kc22jrzz
  Resolved https://github.com/albu/albumentations to commit 880c1aaed10ab74cfe851496a476c1c34cadcd0f
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=56bb039b3220a365ce8bc309e6a8607ef62fca6eed117641019b4afc89aafeb6
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 376 kB 597 kB/s            
     |████████████████████████████████| 84 kB 660 kB/s            


In [2]:
# !pip install timm

In [29]:
import os
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl, utils
from catalyst.contrib.datasets import MNIST
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations
from albumentations import pytorch as AT
import torch
import torch.nn as nn
import timm
from efficientnet_pytorch import EfficientNet
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchmetrics
from albumentations.core.composition import Compose, OneOf
from albumentations.augmentations.transforms import CLAHE, GaussNoise, ISONoise
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchtoolbox.transform import ImageNetPolicy
import matplotlib.pyplot as plt
img_size = 224

In [84]:
class CFG:
    seed = 42
    model_name = 'efficientnet-b3'
    pretrained = True
    img_size = 224
    num_classes = 7
    lr = 3e-4
    min_lr = 1e-6
    t_max = 20
    num_epochs = 40
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [93]:
class Dataset(BaseDataset):
    def __init__(
            self, 
            data_dir=None,
            augmentation=None, 
    ):
        # self.ids = os.listdir(images_dir)
        # self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        # self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # self.ids = []
        self.images = []
        for emo_dir in os.listdir(data_dir):
            cur_dir = os.path.join(data_dir, emo_dir)
            for file in os.listdir(cur_dir):
                self.images.append((emo_dir, os.path.join(cur_dir, file)))
#         try:
    #         except:
#                 print(data_fer_dir)
        labels_tr = ['car', 'broken car']
        self.map_labels = {name : ind for ind, name in enumerate(labels_tr)}
        self.augmentation = augmentation
    
    def get_images_from_folder(self, path):
        return os.listdir(path)

    def __getitem__(self, i):
        
        # read data
        cur_pair = self.images[i]
        image = cv2.imread(cur_pair[1])
#         if 'train_fer' not in cur_pair[1]:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         else:
#             image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.make_square(image)
        label = cur_pair[0]

        # apply augmentations
        if self.augmentation:
            image = self.augmentation(image=image)['image']
            
        
        return image, self.map_labels[label]
    

    def __len__(self):
        return len(self.images)
    def make_square(self, img):
        sz = max(img.shape[0], img.shape[1])
        left = (sz - img.shape[1]) // 2
        right = sz - img.shape[1] - left
        top = (sz - img.shape[0]) // 2
        bottom = sz - img.shape[0] - top
#         print(top, bottom, left, right)
        return cv2.copyMakeBorder(img, top + 10, bottom + 10, left + 10, right + 10, cv2.BORDER_CONSTANT, None, value=0)

In [94]:
def make_square(img):
    sz = max(img.shape[0], img.shape[1])
    left = (sz - img.shape[1]) // 2
    right = sz - img.shape[1] - left
    top = (sz - img.shape[0]) // 2
    bottom = sz - img.shape[0] - top
#         print(top, bottom, left, right)
    return cv2.copyMakeBorder(img, top + 10, bottom + 10, left + 10, right + 10, cv2.BORDER_CONSTANT, None, value=0)

In [95]:
data_transform = {
    'val': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ]),
    'train': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.HorizontalFlip(0.5),
        albumentations.RGBShift(),
        albumentations.Blur(),
#         albumentations.OneOf([
#             albumentations.GaussNoise(),
#             albumentations.ISONoise()]),
        albumentations.ElasticTransform(),
#         albumentations.CoarseDropout(p=1),
        albumentations.Rotate(always_apply=False, p=1.0, limit=(-10, 10), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
#         albumentations.ShiftScaleRotate(p=0.5),
#         albumentations.RandomBrightnessContrast(p=0.5),
#         ImageNetPolicy,
        
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ]),
    'test': albumentations.Compose([
        albumentations.Resize(img_size, img_size),
        albumentations.Normalize(),
#         albumentations.ToFloat(),
#         albumentations.ToGray(always_apply=True, p=1.0),
        AT.ToTensorV2()
        ])
}



In [96]:
# helper function for data visualization
def visualize(images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    cur = 0
    for i, (image, name) in enumerate(images):
        plt.title(name)
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(image.numpy().transpose(1, 2, 0))
        cur += 1
            
    plt.show()

In [97]:
class CustomResnext(nn.Module):
    def __init__(self, model_name='resnext50d_32x4d', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=2)
#         

    def forward(self, x):
        x = self.model(x)
#         x = torch.squeeze(x)
        return x


In [36]:
from efficientnet_pytorch import EfficientNet

In [98]:
# model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)
model = CustomResnext()
# model.load_state_dict(torch.load('./logs/train.7.pth')['model_state_dict'])
# model.to("cuda")

In [38]:
# model

In [99]:
# from catalyst.contrib.schedulers import MultiStepLR, ReduceLROnPlateau

criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
train_data = Dataset(data_dir='../input/dataset-dtp-with-evacuator/dataset_dtp_with_broken_car/dataset_dtp_with_broken_car/train', augmentation=data_transform['train'])
valid_data = Dataset(data_dir='../input/dataset-dtp-with-evacuator/dataset_dtp_with_broken_car/dataset_dtp_with_broken_car/test', augmentation=data_transform['test'])
loaders = {
    "train": DataLoader(train_data, batch_size=16, shuffle=True),
    "valid": DataLoader(valid_data, batch_size=16),
}

In [40]:
!nvidia-smi

Sun Jan 16 16:33:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    35W / 250W |   4033MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [41]:
# optimizer.param_groups[0]['lr'] = 1e-5

In [100]:
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    num_epochs=15,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=[1]),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=2),
        dl.SchedulerCallback(loader_key="valid", metric_key="loss"),
        dl.CheckpointCallback(
            logdir="./logs", loader_key="valid", metric_key="loss", minimize=True
        ),
    ],
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
#     minimize_valid_metric=True,
    verbose=True,
    load_best_on_end=True,
)


1/15 * Epoch (train):   0%|          | 0/193 [00:00<?, ?it/s]

train (1/15) accuracy01: 0.8266102797657776 | accuracy01/std: 0.10366028007800646 | f1/_macro: 0.8254785429200715 | f1/_micro: 0.8266052797960213 | f1/_weighted: 0.826682828652401 | loss: 0.3949531780176528 | loss/mean: 0.3949531780176528 | loss/std: 0.1954055228331429 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.8252180368678974 | precision/_micro: 0.8266102797657775 | precision/_weighted: 0.8268160895854642 | recall/_macro: 0.8258003919068464 | recall/_micro: 0.8266102797657775 | recall/_weighted: 0.8266102797657775


1/15 * Epoch (valid):   0%|          | 0/21 [00:00<?, ?it/s]

valid (1/15) accuracy01: 0.9343283582089552 | accuracy01/std: 0.08079366564881865 | f1/_macro: 0.9336798694815929 | f1/_micro: 0.9343233582357122 | f1/_weighted: 0.9339723709955471 | loss: 0.16802581148361095 | loss/mean: 0.16802581148361095 | loss/std: 0.13223655365769094 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.9411442749762826 | precision/_micro: 0.9343283582089552 | precision/_weighted: 0.9391395935741274 | recall/_macro: 0.9317857142857142 | recall/_micro: 0.9343283582089552 | recall/_weighted: 0.9343283582089551
* Epoch (1/15) lr: 0.0003 | momentum: 0.9


2/15 * Epoch (train):   0%|          | 0/193 [00:00<?, ?it/s]

...

valid (14/15) accuracy01: 0.9552238834438038 | accuracy01/std: 0.058550986887294876 | f1/_macro: 0.9551165069166784 | f1/_micro: 0.9552188806231866 | f1/_weighted: 0.9552124823083937 | loss: 0.1390220918094934 | loss/mean: 0.1390220918094934 | loss/std: 0.17876854533399034 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.9552601486563751 | precision/_micro: 0.9552238805970149 | precision/_weighted: 0.9552281474275278 | recall/_macro: 0.955 | recall/_micro: 0.9552238805970149 | recall/_weighted: 0.9552238805970148
* Epoch (14/15) lr: 0.0003 | momentum: 0.9


15/15 * Epoch (train):   0%|          | 0/193 [00:00<?, ?it/s]

train (15/15) accuracy01: 0.9495770982433315 | accuracy01/std: 0.05392034183096715 | f1/_macro: 0.9491712460799364 | f1/_micro: 0.9495720982696584 | f1/_weighted: 0.9495588834040812 | loss: 0.1226374291043508 | loss/mean: 0.1226374291043508 | loss/std: 0.10425043418419015 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.9494200183736268 | precision/_micro: 0.9495770982433311 | precision/_weighted: 0.9495670718686693 | recall/_macro: 0.9489491224740878 | recall/_micro: 0.9495770982433311 | recall/_weighted: 0.9495770982433311


15/15 * Epoch (valid):   0%|          | 0/21 [00:00<?, ?it/s]

valid (15/15) accuracy01: 0.9164179104477612 | accuracy01/std: 0.08270685358754275 | f1/_macro: 0.9155939207346389 | f1/_micro: 0.9164129104750413 | f1/_weighted: 0.9159661954241752 | loss: 0.20986210270993302 | loss/mean: 0.20986210270993302 | loss/std: 0.21635564807933425 | lr: 0.0003 | momentum: 0.9 | precision/_macro: 0.9228088739692039 | precision/_micro: 0.9164179104477612 | precision/_weighted: 0.9209291788158384 | recall/_macro: 0.9138392857142856 | recall/_micro: 0.9164179104477612 | recall/_weighted: 0.916417910447761
* Epoch (15/15) lr: 0.0003 | momentum: 0.9
Top best models:
logs/train.10.pth	0.0935


In [101]:
torch.save(model, 'resnext_car4_with_broken_car.pth')
torch.save(model.state_dict(), 'resnext_car_dict4_with_broken_car.pth')

In [44]:
torch.cuda.empty_cache()

In [45]:
# mod = runner.bes

In [46]:
# import os
# os.listdir('../input/emotion-recognition-dataset/RDFKDEF_yolo_bounding_box30px/rfd_cropped/test_many_photos_per_human')

In [47]:

# model1 = timm.create_model('resnet152', pretrained=True, num_classes=2)
# # print(torch.load('./logs/train.7.pth'))

In [48]:
# pretrained_model = MyEffnet()
# # pretrained_model = torch.load('./logs/train.7.pth')
# pretrained_model.load_state_dict(torch.load('./logs/train.7.pth')['model_state_dict'])
# pretrained_model.to('cuda')
# pretrained_model.eval()

In [49]:
# model.cuda()
# model = EmotionsLightning(MyEffnet())
# model = EmotionsLightning(model=MyEffnet()).load_from_checkpoint(checkpoint_callback.best_model_path, model=MyEffnet())
# model.to('cuda')
# model = MyEffnet()
# model_cfg = torch.load(checkpoint_callback.best_model_path)
# checkpoint_callback.best_model_path
# model.eval()
# model = MyEffnet()
# model.load_state_dict(checkpoint_callback.best_model_path, model_cfg['state_dict'])
# model_cfg['state_dict']
# # print(model)
# model.to('cuda')

In [50]:
# model.load_state_dict(utils.get_averaged_weights_by_path_mask(logdir="./logs", path_mask="*.pth"))

In [51]:
torch.save(model, 'best_model.pth')
torch.save(model.state_dict(), 'best_model_dict.pth')